# HDS5210-2022 Midterm

In the midterm, you're going to use all the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged given their risk and various characteristics of the patient.

Each step in the midterm will build up to form your complete solution.

**Make sure you write good docstrings and doctests along the way!!**

**The midterm is due at 11:59 PM CST on Thursday October 24.**

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex (Gender assigned at birth)
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should return None.

NOTES:
1. In the final step there is a table that translates from **PRIEST Score** to **30-day probability of an outcome** but the last two probabilities are shown as ranges (59-88% and >99%).  Our code needs to output a single number, however. For our code, use the following rule:
 * If PRIEST score is between 17 and 25, the probability you return should be 0.59
 * If PRIEST score is greater than or equal to 26, the probability you return should be 0.99


In [16]:
def priest(sex, age, respiratory_rate, o2sat, heart_rate, systolic_bp, temp, alertness, inspired_o2, status):
  """
  Calculate the PRIEST Clinical Severity Score for a patient based on input parameters.

  Parameters:
  - sex (str): The patient's gender assigned at birth ('male' or 'female').
  - age (int): The patient's age in years.
  - respiratory_rate (int): The patient's respiratory rate in breaths per minute.
  - o2sat (float): The patient's oxygen saturation as a percentage between 0 and 1.
  - heart_rate (int): The patient's heart rate in beats per minute.
  - systolic_bp (int): The patient's systolic blood pressure in mmHg.
  - temp (float): The patient's temperature in degrees Celsius.
  - alertness (str): The patient's alertness as a string description ('alert', 'confused', or 'not alert').
  - inspired_o2 (str): The patient's inspired oxygen as a string description ('Air' or 'Supplemental oxygen').
  - status (str): The patient's performance status as a string description ('Unrestricted normal activity', 'Limited strenuous activity, can do light activity', 'Limited activity, can self-care', 'Limited self-care', or 'Bed/chair bound, no self-care').

  Returns:
  - float: Risk score calculation based on PRIEST score, between 0 and 1. Will return None for invalid inputs.

  Note:
  - If PRIEST score is between 17 and 25, the probability will be 0.59.
  - If PRIEST score is greater than or equal to 26, the probability will be 0.99.
  """

  sex = sex.lower()
  if sex not in ['male','female']:
    return None

  try:
    age = int(age)
    respiratory_rate = int(respiratory_rate)
    o2sat = float(o2sat)
    heart_rate = int(heart_rate)
    systolic_bp = int(systolic_bp)
    temp = float(temp)
  except (ValueError, TypeError):
    return None

  # Intialized PRIEST score
  priest = 0

  #Calculation of PRIEST score

  # Sex
  if sex == 'male':
    priest += 1

  if sex == 'female':
    priest += 0

  # Age
  if 16 <= age <= 49:
    priest += 0
  elif 50 <= age <= 65:
    priest += 2
  elif 66<= age <= 80:
    priest += 3
  elif age >= 81:
    priest += 4

  # Respiratory rate
  if respiratory_rate <= 9:
    priest += 3
  elif 9 <= respiratory_rate <= 11:
    priest += 1
  elif 12 <= respiratory_rate <= 20:
    priest += 0
  elif 21 <= respiratory_rate <= 23:
    priest += 2
  elif respiratory_rate >= 24:
    priest += 3

  # Oxygen saturation
  if o2sat >= 0.95:
    priest += 0
  elif 0.94 <= o2sat <= 0.95:
    priest += 1
  elif 0.92 <= o2sat <= 0.93:
    priest += 2
  elif o2sat <= 0.92:
    priest += 3

  # Heart rate
  if heart_rate <= 41:
    priest += 3
  elif 41 <= heart_rate <= 50:
    priest += 1
  elif 51 <= heart_rate <= 90:
    priest += 0
  elif 91 <= heart_rate <= 110:
    priest += 1
  elif 111 <= heart_rate <= 130:
    priest += 2
  elif heart_rate >= 131:
    priest += 3

  # Systolic BP
  if systolic_bp <= 91:
    priest += 3
  elif 91 <= systolic_bp <= 100:
    priest += 2
  elif 101 <= systolic_bp <= 110:
    priest += 1
  elif 111 <= systolic_bp <= 219:
    priest += 0
  elif systolic_bp >= 220:
    priest += 3

  # Temperature
  if temp <= 35.1:
    priest += 3
  elif 35.1 <= temp <= 36.0:
    priest += 1
  elif 36.1 <= temp <= 38.0:
    priest += 0
  elif 38.1 <= temp <= 39.0:
    priest += 1
  elif temp >= 39:
    priest += 2


  #Alertness
  alertness = alertness.lower()
  if alertness == 'alert':
    priest += 0
  elif alertness == 'confused' or alertness == "not alert":
    priest += 3


  #Inspired oxygen
  inspired_o2 = inspired_o2.lower()
  if inspired_o2 == 'air':
    priest += 0
  elif inspired_o2 == 'supplemental oxygen':
    priest += 2

  #Performance status
  status = status.lower()
  if status == 'unrestricted normal activity':
    priest += 0
  elif status == 'limited strenuous activity, can do light activity':
    priest += 1
  elif status == 'limited activity, can self-care':
    priest += 2
  elif status == 'limited self-care':
    priest += 3
  elif status == 'bed/chair bound, no self-care':
    priest += 4

  print(f"Calculated PRIEST score: {priest}")

  #Risk percentage based on PRIEST score

  if priest <= 1:
      return 0.01
  elif priest == 2 or priest == 3:
      return 0.02
  elif priest == 4:
      return 0.03
  elif priest == 5:
      return 0.09
  elif priest == 6:
      return 0.15
  elif priest == 7:
      return 0.18
  elif priest == 8:
      return 0.22
  elif priest == 9:
      return 0.26
  elif priest == 10:
      return 0.29
  elif priest == 11:
      return 0.34
  elif priest == 12:
      return 0.38
  elif priest == 13:
      return 0.46
  elif priest == 14:
      return 0.47
  elif priest == 15:
      return 0.49
  elif priest == 16:
      return 0.55
  elif 17 <= priest <= 25:
      return 0.59
  elif priest >= 26:
      return 0.99
if __name__ == "__main__":
    import doctest
    doctest.testmod()

print(priest('male', 75, 32, 0.85, 130, 85, 39.5, 'Alert', 'Air', 'limited strenuous activity, can do light activity'))
print(priest('FeMale', 63, 24, 0.92, 110, 210, 36.3, 'Confused', 'Supplemental oxygen','limited self-care '))
print(priest('MALE', 22, 19, 0.90, 72, 180, 36.5, 'ALERT', 'AIR', 'UNRESCRICTED NORMAL ACTIVITY' ))

Calculated PRIEST score: 18
0.59
Calculated PRIEST score: 13
0.46
Calculated PRIEST score: 4
0.03


In [17]:
import doctest
doctest.run_docstring_examples(priest, globals(),verbose=True)

Finding tests in NoName


## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

My function is not smart enough to understand `'MALE'` is the same as `'male'`.  You have to send it exactly `'male'` or `'female'`

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service using the `requests` module
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None
5. Include a good docstring with at least three test cases.


In [30]:
import requests
def find_hospital(age, sex, risk_pct):

  base_url = "https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/"

  parameters = {
      "age": age,
      "sex": sex,
      "risk_pct": risk_pct
  }

  try:
    response = requests.get(base_url, params=parameters)

    response.raise_for_status ()

    data = response.json()

    return data.get("hospital")

  except (requests.RequestException, ValueError, KeyError):
    return None

if __name__ == "__main__":
  import doctest
  doctest.run_docstring_examples(find_hospital, globals(), verbose=True)

print(find_hospital(75, 'male', 0.59))
print(find_hospital(63,"FeMale", 0.46))
print(find_hospital(17, "Male", 0.2))

Finding tests in NoName
Wesley Woods Geriatric Hospital
Emory Dunwoody Medical Center
Childrens Healthcare of Atlanta at Scottish Rite


In [19]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(),verbose=True)

Finding tests in NoName


## Part 3: Get the address for that hospital from a JSON file

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list from this webpage to lookup the address for that hospital, based on its name.  https://www.officialusa.com/stateguides/health/hospitals/georgia.html

Because we skipped the section about Beautiful Soup and working with HTML, I've converted this information into a JSON document for you.  It's available for you here.  Your code should retrieve this file using the `requests` module.

`https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-`

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data from this JSON file for the hospital you want to find.
2. Your code will have to load the JSON and return the correct hospital based on name.
3. If the hospital name isn't found, the function should return None.
4. Be sure to use good docstring documentation and includes at least 3 test cases.

In [44]:
import requests
import json

def get_address(hospital_name):

  json_url = "https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-"

  try:
    response = requests.get(json_url)
    response.raise_for_status()
    hospital_data = response.json()


    if isinstance(hospital_data, list):
      for hospital in hospital_data:
        if isinstance(hospital, dict):
           print(f"Hospital Name: {hospital.get('name')}")
           if hospital.get("name").strip().lower() == hospital_name.strip().lower():
              return hospital.get("address")

    return None

  except (requests.RequestException, json.JSONDecodeError) as e:
        print(f"Error occurred: {e}")
        return None

if __name__ == "__main__":
  import doctest
  doctest.run_docstring_examples(get_address, globals(), verbose=True)

print(get_address("EMORY UNIVERSITY HOSPITAL"))

Finding tests in NoName
None


## Part 4: Run the risk calculator on a population

At the link below, there is a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.  Your code should use the `requests` module to retrieve the file from this URL.

`https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh`


In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes the file location above as its only parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like this: `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`.  Look at the file in Part 5 for what the output looks like.
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you withe file in Part 5.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` at the link below.  Write some code to check your results.  This does not need to be a function.

`https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5`


---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

Submit your work as usual into a folder named `midterm`

---